In [1]:
import os
import re
import getpass
from typing import List, Dict
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Redis
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
    AttributeInfo)
from dotenv import load_dotenv
load_dotenv(override=True)
import sys 
sys.path.append("..")
from utils.database_managers import QDrantDBManager
from utils.embedding import EmbeddingFunction
from utils.text_processing import TextSplitter
from utils.language_models import LangChainAI
from langchain_groq import ChatGroq

In [2]:
text_splitter = TextSplitter(
    chunk_size=2000, 
    chunk_overlap=20
)
embedding = EmbeddingFunction('openAI').embedder
QDRANT_URL=os.getenv('QDRANT_URL')
COLLECTION_NAME="csv-openAI-embed"

qdrantClient = QDrantDBManager(
    url=QDRANT_URL,
    port=6333,
    collection_name=COLLECTION_NAME,
    vector_size=1536,
    embedding=embedding,
    record_manager_url="sqlite:///record_manager_cache.sql"
)
retriever = qdrantClient.vector_store.as_retriever()

c:\Users\ELAFACRB1\Codice\GitHub\riassume\riassume\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Collection csv-openAI-embed already exists!


c:\Users\ELAFACRB1\Codice\GitHub\riassume\riassume\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [3]:
llm = ChatGroq(model_name="llama3-70b-8192", groq_api_key=os.getenv('GROQ_API_KEY'), temperature=0)

In [7]:
def print_result(response_obj):
    print("SOURCES: \n")
    cnt = 1
    for source_doc in response_obj["source_documents"]:
        print(f"Prodotto #{cnt}")
        cnt += 1
        # print("Source Metadata: ", source_doc.metadata)
        print("Descrizione:")
        print(source_doc.page_content)
        print("\n")
    print("RESULT: \n")
    print(response_obj["result"] + "\n\n")

In [5]:
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    # we'll want to see source documents for comparison
    return_source_documents=True
)

In [6]:
# query = "puoi consigliarmi un buon monitor da gaming da 25"" ? "
# response = retriever({"query":query})
# print_result(response)

c:\Users\ELAFACRB1\Codice\GitHub\riassume\riassume\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


SOURCES: 

Chunk #1
Descrizione:
Nome: Dell Alienware AW2523HF 25"
Descrizione: Monitor gaming da 25 pollici con pannello TN FHD (1920 x 1080)
Link: https://www.hp.com/it-it/shop/product.aspx?id=780J4AA&opt=ABB&sel=MTO&gad_source=1&gclid=Cj0KCQjwsuSzBhCLARIsAIcdLm43Fk3Y2DeltVRzAZ3QH-3EBAKge7XxGyVeF9-xZyPA6XtzsT6kvNcaAneUEALw_wcB&gclsrc=aw.ds


Chunk #2
Descrizione:
Nome: LG 27GP950-B 27"
Descrizione: Monitor gaming da 27 pollici con pannello Nano IPS QHD (2560 x 1440)
Link: https://www.lg.com/it/monitor/gaming/27gp850p-b-baeu/?gad_source=1&gclid=Cj0KCQjwsuSzBhCLARIsAIcdLm7jazdjzrHL8Zs_eVzIKGcMgbIoomdq30onozCwN70Uup7iknXV0D8aAhtxEALw_wcB&gclsrc=aw.ds


Chunk #3
Descrizione:
Nome: Acer Nitro XV282K KVbmiix 28"
Descrizione: Monitor gaming da 28 pollici con pannello IPS UHD (3840 x 2160)
Link: https://www.mediaworld.it/it/product/_acer-nitro-xv275up3biiprx-27-pollici-2560-x-1440-pixel-212691.html?utm_source=trovaprezzi&utm_medium=psm-product%20feed&utm_campaign=rt_pricecomp_na_sp_na_always